In [ ]:
import ckanapi
import requests
import json
import zipfile
import os

In [ ]:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
package_id = "bike-share-toronto-ridership-data"

# Get package information
package_url = f"{base_url}/api/3/action/package_show"
params = {"id": package_id}
package = requests.get(package_url, params=params).json()

Download

In [ ]:
for resource in package["result"]["resources"]:
    if not resource["datastore_active"]:
        resource_id = resource["id"]
        resource_name = resource["name"]
        resource_url = resource["url"]
        # print(resource_name, resource_url)
        if 'bikeshare-ridership-2022' in resource_name:  # Use 'in' for substring check
          response = requests.get(resource_url)
          if response.status_code == 200:
              file_extension = resource["format"].lower()
              filename = f"{resource_name}.{file_extension}"

              with open(filename, "wb") as file:
                  file.write(response.content)
              print(f"Saved as: {filename}\n")
          else:
              print(f"Failed to download {resource_name}\n")

Extract

In [ ]:
def extract_zip(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(zip_path)  # Delete the zip file after extraction

def process_directory(directory):
    for root, dirs, files in os.walk(directory, topdown=False):  # Ensure directories are processed last
        for file in files:
            file_path = os.path.join(root, file)
            if file.endswith('.zip'):
                print(f"Extracting {file_path}")
                extract_zip(file_path, root)
        
        # Ensure only CSV files remain
        for file in os.listdir(root):
            file_path = os.path.join(root, file)
            if os.path.isdir(file_path):
                continue  # Skip directories
            if not file.endswith('.csv'):
                print(f"Removing non-CSV file: {file_path}")
                os.remove(file_path)

In [ ]:
zip_file_path = "bikeshare-ridership-2022.zip"
extract_location = "./"

# if not os.path.exists(extract_location):
#     os.makedirs(extract_location)

extract_zip(zip_file_path, extract_location)
process_directory(extract_location)